In [2]:
import requests
import urllib3
urllib3.disable_warnings()
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import pandas as pd

In [3]:
def preliminary_create_website_list(lst_title):
    website_list = []

    for i in lst_title:
        res = ""
        i = i.replace('Pty Ltd', '').replace('- authorised electricity retailer', '').lower().strip().replace(' ', '')
        res = i + '.com.au'
        website_list.append(res)

    return website_list


In [3]:
def preliminary_conclude(ori_title_list, new_title_list, df_ori, df):
    new_added = []
    old_gone = []

    for i in ori_title_list:
        if i not in new_title_list and i != 'Title':
            old_gone.append(i)
    for i in new_title_list:
        if i not in ori_title_list:
            new_added.append(i)

    print('新增:', ",\n ".join(new_added), f'\n 一共{len(new_added)}家')
    print('减少:', ",\n ".join(old_gone), f'\n 一共{len(old_gone)}家')

In [58]:
def access_URL(url = 'https://www.aer.gov.au/retail-markets/authorisations/public-register-of-authorised-retailers-authorisation-applications?'):
    df = pd.DataFrame()

    url_list = []
    for i in range(0,10):
        headers = {
            'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ' \
                            'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
        }
        paras = {
            'page':i
        }

        #1.url
        url1 = url + urlencode(paras)
        url_list.append(url1)

    for i in url_list:
        temp0 = pd.read_html(i)
        temp = pd.DataFrame(temp0[0])
        df = df.append(temp)
    return df

In [59]:
df = access_URL()

In [64]:
df2 = df.loc[(df.Status == "Historical") & (df.Sector != 'Gas')]
df1 = df.loc[(df.Sector == 'Electricity') & ((df.Status == 'Current') | (df.Status == 'Open'))]
df3 = df1.append(df2, ignore_index = True)

df_ori = pd.read_excel('澳大利亚电力零售商（已授权）.xlsx')

ori_title_list = list(df_ori['Title'])
new_title_list = list(df3['Title'])

print(f'从 AER 中下载 Public register of authorised retailers & authorisation applications, 一共 {len(df) - 1} 家, 其中有{len(df2)}的 status 为 historical')
website_list = preliminary_create_website_list(new_title_list)

df3['Website'] = pd.DataFrame(website_list)

df3.to_excel('preliminary_res.xlsx')


从 AER 中下载 Public register of authorised retailers & authorisation applications, 一共 181 家, 其中有16的 status 为 historical


In [24]:
# import pandas as pd
df_new = pd.read_excel('澳大利亚电力零售商（已授权）1.xlsx')
df_ori = pd.read_excel('澳大利亚电力零售商（已授权）.xlsx')
ori_title_list = list(df_ori['Title'])
new_title_list = list(df_new['Title'])
preliminary_conclude(ori_title_list, new_title_list, df_ori, df_new)


新增: ARC Utilities Management Pty Ltd - authorised electricity retailer,
 Accel Energy Retail Pty Ltd - Application for electricity retailer authorisation,
 Tilt Renewables Retail Pty Ltd retailer electricity authorisation,
 ERC Energy Pty Ltd - Application for electricity retailer authorisation,
 Ampol Energy (Retail) Pty Ltd - Application for electricity retailer authorisation,
 iGENO Pty Limited - application for electricity retailer authorisation,
 Mojo Power East (trading as People Energy) - authorised electricity retailer,
 3P Energy Pty Ltd - application for electricity retailer authorisation - withdrawn,
 Rush Energy Pty Ltd - application for electricity retailer authorisation - withdrawn,
 CO Zero - authorised electricity retailer - revoked,
 Urth Energy Pty Ltd - authorised electricity retailer - revoked,
 Ultium Energy Pty Ltd - application for electricity retailer authorisation,
 GoEnergy - authorised electricity retailer - revoked,
 Australian Power and Gas - authorised ele